## Import

In [48]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [4]:
# seeds
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [5]:
import torch
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

## Hyperparameter Setting

In [6]:
# hyperparameter
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':25,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':42
}

## Fixed RandomSeed

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [8]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [9]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "new_train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)

In [10]:
# # folder name list
ori_names = os.listdir('ori_train')

new_names = ['furniture_repair', 'cleaning_mop_holder_repair', 'mold', 'twist', 'rust_contamination', 'wobbling', 'fabric_defect', 'molding_repair', 'stain', 'plaster_repair', 'pollution', 'typographical_error', 'crying', 'defective_joint', 'windowdoor_frame_repair', 'cracking', 'excessive_gap', 'piece', 'damage']

# print(len(ori_names), len(new_names))
# os.chdir('new_train')
# # # folder rename
# for ori_name, new_name in zip(ori_names, new_names):
#     os.rename(ori_name, new_name)

# print(os.listdir())
# os.chdir('../')

In [11]:
for a,b in zip(ori_names, new_names):
    len_a = len(os.listdir(f'ori_train/{a}'))
    len_b = len(os.listdir(f'new_train/{b}'))
    if len_a == len_b:
        print('go on')
    else: 
        print('stop')

go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on


In [12]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [13]:
df

,img_path,label
0,new_train\cleaning_mop_holder_repair\0.png,cleaning_mop_holder_repair
1,new_train\cleaning_mop_holder_repair\1.png,cleaning_mop_holder_repair
2,new_train\cleaning_mop_holder_repair\10.png,cleaning_mop_holder_repair
3,new_train\cleaning_mop_holder_repair\100.png,cleaning_mop_holder_repair
4,new_train\cleaning_mop_holder_repair\101.png,cleaning_mop_holder_repair
...,...,...
3452,new_train\wobbling\53.png,wobbling
3453,new_train\wobbling\6.png,wobbling
3454,new_train\wobbling\7.png,wobbling
3455,new_train\wobbling\8.png,wobbling


In [14]:
# from sklearn.model_selection import train_test_split

# train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])


## Label-Encoding

In [15]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df


,img_path,label
0,new_train\cleaning_mop_holder_repair\0.png,0
1,new_train\cleaning_mop_holder_repair\1.png,0
2,new_train\cleaning_mop_holder_repair\10.png,0
3,new_train\cleaning_mop_holder_repair\100.png,0
4,new_train\cleaning_mop_holder_repair\101.png,0
...,...,...
3452,new_train\wobbling\53.png,18
3453,new_train\wobbling\6.png,18
3454,new_train\wobbling\7.png,18
3455,new_train\wobbling\8.png,18


In [16]:
train = df[df['label']==14]

In [17]:
train

,img_path,label
3021,new_train\stain\0.png,14
3022,new_train\stain\1.png,14
3023,new_train\stain\2.png,14


## CustomDataset

In [18]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
            image = image.unsqueeze(0)
            
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [19]:

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2



train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2() 
])



# test_transform = A.Compose([
#     A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2()
# ])


In [20]:
from PIL import Image

image_path = 'new_train\\stain\\0.png'

with Image.open(image_path) as img:
    width, height = img.size

print(f"Image size: {width} x {height}")


Image size: 618 x 697


In [21]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [26]:
import torchvision.utils as vutils


# Step 2: Define the generator and discriminator models
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), 100, 1, 1) # Reshape noise tensor to (batch_size, 100, 1, 1)
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        # Reshape the input tensor to have a shape of [batch_size, num_channels, height, width]
        input = input.view(-1, 3, 224, 224)
        return self.main(input)
    
# Step 3: Set up the loss functions and optimizers
ngpu = 1
generator = Generator().to(device)
discriminator = Discriminator(ngpu).to(device)


criterion = nn.BCELoss()

lr = 0.0002
optim_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optim_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Step 4: Create the training loop
num_epochs = 20
real_label = 1
fake_label = 0

output_dir = "output"
checkpoint_dir = "checkpoints"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

fixed_noise = torch.randn(64, 100, 1, 1, device=device)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)
        batch_size = images.size(0)

        # Train the discriminator
        optim_D.zero_grad()
        real_labels = torch.full((batch_size,), real_label, device=device)
        output_real = discriminator(images).view(-1).squeeze()
        errD_real = criterion(output_real, real_labels)

        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_images = generator(noise)
        fake_labels = torch.full((batch_size,), fake_label, device=device)
        output_fake = discriminator(fake_images.detach()).view(-1).squeeze()
        errD_fake = criterion(output_fake, fake_labels)

        errD = errD_real + errD_fake
        errD.backward()
        optim_D.step()

        # Train the generator
        optim_G.zero_grad()
        real_labels = torch.full((batch_size,), real_label, device=device)
        output = discriminator(fake_images).view(-1).squeeze()
        errG = criterion(output, real_labels)

        errG.backward()
        optim_G.step()

        # Print losses
        if i % 50 == 0:
            print(f"[{epoch}/{num_epochs}] [{i}/{len(train_loader)}] Loss_D: {errD.item()} Loss_G: {errG.item()}")

        # Save generator's output periodically
        if i % 500 == 0:
            with torch.no_grad():
                fake_images = generator(fixed_noise).detach().cpu()
            img_list = vutils.make_grid(fake_images, padding=2, normalize=True)
            vutils.save_image(img_list, os.path.join(output_dir, f"fake_images_epoch_{epoch}_iter_{i}.png"))

    # Save model checkpoints
    torch.save(generator.state_dict(), os.path.join(checkpoint_dir, f"generator_epoch_{epoch}.pth"))
    torch.save(discriminator.state_dict(), os.path.join(checkpoint_dir, f"discriminator_epoch_{epoch}.pth"))

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b2(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [22]:
import torch.backends.cudnn as cudnn
cudnn.enabled = False

In [23]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.long).to(device)
        
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [24]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.long).to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [25]:
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 22/22 [00:06<00:00,  3.45it/s]


Epoch [1], Train Loss : [1.29716] Val Loss : [0.89246] Val Weighted F1 Score : [0.70932]


100%|██████████| 22/22 [00:06<00:00,  3.48it/s]


Epoch [2], Train Loss : [0.56121] Val Loss : [0.82680] Val Weighted F1 Score : [0.74148]


100%|██████████| 22/22 [00:06<00:00,  3.34it/s]


...

Epoch [23], Train Loss : [0.00403] Val Loss : [1.13965] Val Weighted F1 Score : [0.80456]
Epoch 00023: reducing learning rate of group 0 to 1.8750e-05.


100%|██████████| 22/22 [00:06<00:00,  3.44it/s]


Epoch [24], Train Loss : [0.00292] Val Loss : [1.15358] Val Weighted F1 Score : [0.80239]


100%|██████████| 22/22 [00:06<00:00,  3.39it/s]

Epoch [25], Train Loss : [0.00397] Val Loss : [1.15605] Val Weighted F1 Score : [0.80327]


In [2]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

0

## Inference

In [28]:
test = pd.read_csv('test.csv')
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [29]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [30]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [31]:
preds = inference(infer_model, test_loader, device)

100%|██████████| 25/25 [00:07<00:00,  3.44it/s]


## Submission

In [32]:
submit = pd.read_csv('sample_submission.csv')

In [33]:
submit['label'] = preds

In [34]:
submit

,id,label
0,TEST_000,damage
1,TEST_001,pollution
2,TEST_002,damage
3,TEST_003,molding_repair
4,TEST_004,pollution
...,...,...
787,TEST_787,damage
788,TEST_788,damage
789,TEST_789,pollution
790,TEST_790,pollution


In [35]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [36]:
# Dictionary to map old column names to new column names
name_map = dict(zip(new_names, ori_names))
name_map


# Rename columns using the rename() method
submit_test = submit.replace(name_map)
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [37]:
submit_test.to_csv(os.path.join(Record_path,'submission.csv'), index=False)

In [38]:
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [40]:
# model information 저장
import json
model_info = {'CFG' : CFG, 'model' : str(model), 'optimizer' : str(optimizer), 'scheduler' : str(scheduler), 'train_serial' : train_serial}
with open(os.path.join(Record_path,'model_info.json'), 'w') as f:
    json.dump(model_info, f)